In [44]:
import pandas as pd
import math
import numpy as np
from geopy.distance import geodesic
from itertools import combinations
from tqdm import tqdm
import random

# **Solution**

In [45]:
# Function to find the closest unvisited city using geodesic distance
def find_closest_city(data, start_coord, visited_cities):
    # Initialize variables to store the closest city
    min_distance = float('inf')
    closest_city = None
    closest_city_coord = None
    
    # Loop through each city to calculate the geodesic distance
    for index, row in data.iterrows():
        city_name = row[0]
        city_coord = (row[1], row[2])  
        
        # Skip the city if it's already visited
        if city_name in visited_cities:
            continue
        
        # Calculate geodesic distance between start_coord and city_coord
        distance = geodesic(start_coord, city_coord).kilometers
        
        # Update the closest city if this one is closer
        if distance < min_distance and distance != 0:
            min_distance = distance
            closest_city = city_name
            closest_city_coord = city_coord
    
    return closest_city, closest_city_coord, min_distance
    

# Function to solve the TSP using geodesic distance and nearest neighbor
def tsp_nearest_neighbor(data, start_city, start_city_coord):
    
    # Initialize visited cities array
    visited_cities = []
    
    # Add starting city to the visited list
    visited_cities.append(start_city)
    
    #print(f"Starting city: {start_city}")
    
    total_distance = 0
    current_city_coord = start_city_coord
    
    # Loop until all cities are visited
    with tqdm(total=len(data) - 1, desc="Visiting cities") as pbar:
        while len(visited_cities) < len(data):
            # Find the closest unvisited city
            closest_city, closest_city_coord, min_distance = find_closest_city(data, current_city_coord, visited_cities)
            
            # Add the closest city to the visited cities array
            visited_cities.append(closest_city)
            
            # Update the current city coordinates to the new closest city's coordinates
            current_city_coord = closest_city_coord
            
            # Accumulate the total distance
            total_distance += min_distance
            
            #print(f"Travels to {closest_city} with a distance of {min_distance:.2f} km")
            
            # Update the progress bar
            pbar.update(1)
    
    # Return to the start city to complete the loop
    return_to_start_distance = geodesic(current_city_coord, start_city_coord).kilometers
    total_distance += return_to_start_distance
    visited_cities.append(start_city)  # Add the starting city at the end to close the loop
    
    #print(f"Returning to {start_city} with a distance of {return_to_start_distance:.2f} km")
    
    return visited_cities, total_distance

In [66]:
# Import CSV file
def import_cities(filename):
    return pd.read_csv(filename)

# Choose a random starting city
def choose_random_city(data):
    random_index = np.random.choice(data.index)
    city_name = data.iloc[random_index][0]
    city_coord = (data.iloc[random_index][1], data.iloc[random_index][2])  
    return city_name, city_coord

In [ ]:
#data = import_cities('vanuatu.csv')
#data = import_cities('italy.csv')
#data = import_cities('us.csv')
#data = import_cities('russia.csv')
data = import_cities('china.csv')
start_city, start_city_coord = choose_random_city(data)

/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_58327/1238140414.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_name = data.iloc[random_index][0]
/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_58327/1238140414.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_coord = (data.iloc[random_index][1], data.iloc[random_index][2])


In [72]:
# print of solution
visited_cities, total_distance = tsp_nearest_neighbor(data, start_city, start_city_coord)
print('------------------------------------------------------------')
print(f"Total distance traveled: {total_distance:.2f} km")
travel_order = " -> ".join(visited_cities)
print(f"Travel order: {travel_order}")

Visiting cities:   0%|          | 0/165 [00:00<?, ?it/s]/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_58327/571662069.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_name = row[0]
/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_58327/571662069.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_coord = (row[1], row[2])
Visiting cities: 100%|██████████| 165/165 [00:02<00:00, 61.13it/s] 

------------------------------------------------------------
Total distance traveled: 42152.87 km
Travel order: Miass -> Zlatoust -> Chelyabinsk -> Kopeysk -> Kamensk‐Uralskiy -> Yekaterinburg -> Pervouralsk -> Nizhniy Tagil -> Perm -> Berezniki -> Izhevsk -> Neftekamsk -> Naberezhnye Chelny -> Nizhnekamsk -> Almetyevsk -> Oktyabrskiy -> Ufa -> Sterlitamak -> Salavat -> Orenberg -> Orenburg -> Novotroitsk -> Orsk -> Magnitogorsk -> Kurgan -> Tyumen -> Tobolsk -> Nefteyugansk -> Surgut -> Nizhnevartovsk -> Noyabrsk -> Novyy Urengoy -> Norilsk -> Seversk -> Tomsk -> Kemerovo -> Leninsk‐Kuznetskiy -> Prokopyevsk -> Novokuznetsk -> Biysk -> Barnaul -> Novosibirsk -> Rubtsovsk -> Omsk -> Achinsk -> Krasnoyarsk -> Kyzyl -> Angarsk -> Irkutsk -> Ulan‐Ude -> Chita -> Bratsk -> Yakutsk -> Magadan -> Petropavlovsk‐Kamchatskiy -> Yuzhno‐Sakhalinsk -> Komsomolsk‐na‐Amure -> Khabarovsk -> Ussuriysk -> Artyom -> Vladivostok -> Nakhodka -> Blagoveshchensk -> Syktyvkar -> Kirov -> Yoshkar‐Ola -> Novoc

## Genetic algorithm 

In [62]:
#filepath = 'italy.csv'
#filepath = 'us.csv'
#filepath = 'russia.csv'
#filepath = 'china.csv'
filepath = 'vanuatu.csv'


CITIES = pd.read_csv(filepath, header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
#CITIES.head()

In [63]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

class Individual:
    def __init__(self, genome):
        self.genome = genome
        self.fitness = -tsp_cost(genome)  

#Parent selection
def parent_selection(population):
    candidates = sorted(random.sample(population, 3), key=lambda e: e.fitness, reverse=True)
    return candidates[0]

#Crossover 
def crossover(p1: Individual, p2: Individual):
    start, end = sorted(random.sample(range(1, len(CITIES)), 2))
    
    #PMX crossover
    child_genome = [None] * len(CITIES)
    child_genome[start:end] = p1.genome[start:end]

    p2_pointer = 0
    for i in range(len(CITIES)):
        if child_genome[i] is None:
            while p2.genome[p2_pointer] in child_genome:
                p2_pointer += 1
            child_genome[i] = p2.genome[p2_pointer]

    child_genome.append(child_genome[0])

    return Individual(child_genome)

#Mutation
def mutation(p: Individual):
    genome = p.genome.copy()

    points = sorted(random.sample(range(1, len(genome) - 1), random.randint(3, 5)))
    segments = [genome[points[i]:points[i + 1]] for i in range(len(points) - 1)]
   
    inverted_segments = [seg[::-1] if random.random() < 0.5 else seg for seg in segments]
    new_genome = genome[:points[0]] + sum(inverted_segments, []) + genome[points[-1]:]

    return Individual(new_genome)

In [64]:
def greedy_nearest_neighbor():
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = 0
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    tsp.append(tsp[0])
    return Individual(tsp)

In [65]:
POPULATION_SIZE = 100  
OFFSPRING_SIZE = 20   

#Population initialization of initial paths found with greedy methods
population = [greedy_nearest_neighbor()]
#Added random individuals to complete the population
for _ in range(POPULATION_SIZE - len(population)):
    random_path = list(range(len(CITIES)))
    random.shuffle(random_path)
    random_path.append(random_path[0])  
    population.append(Individual(random_path))

def evolve(population, generations=200_000, mutation_rate=0.3, initial_temperature=10_000.0, cooling_rate=0.995, stagnation_limit=5_000, adaptive_increase=0.1):
    best_path = max(population, key=lambda ind: ind.fitness)
    current_best = best_path
    temperature = initial_temperature
    stagnation_counter = 0  
    base_mutation_rate = mutation_rate  
    history = []  
    
    for generation in range(generations):
        offspring = []
        for _ in range(OFFSPRING_SIZE):
            if random.random() < mutation_rate:
                parent = parent_selection(population)
                child = mutation(parent)
            else:
                parent1 = parent_selection(population)
                parent2 = parent_selection(population)
                child = crossover(parent1, parent2)
            offspring.append(child)

        improved = False
        for child in offspring:
            child.fitness = -tsp_cost(child.genome)
            delta_fitness = child.fitness - current_best.fitness

            if delta_fitness > 0 or random.random() < np.exp(delta_fitness / temperature):
                current_best = child
                if child.fitness > best_path.fitness:
                    best_path = child
                    improved = True
                    stagnation_counter = 0
        
        if not improved:
            stagnation_counter +=1
        
        if stagnation_counter >= stagnation_limit:
            mutation_rate = min(1.0, mutation_rate + adaptive_increase)

        if improved: 
            mutation_rate = base_mutation_rate

        population.extend(offspring)
        population.sort(key=lambda ind: ind.fitness, reverse=True)
        population = population[:POPULATION_SIZE]

        temperature *= cooling_rate
        
        history.append(-best_path.fitness)

        if generation % 10_000 == 0:
            print(f"Generation {generation}: Cost of the best route: {-best_path.fitness:.2f} km")
            random_path = list(range(len(CITIES)))
            random.shuffle(random_path)
            random_path.append(random_path[0]) 
            population.append(Individual(random_path))
    
    return best_path, history

best_path, history = evolve(population)


print(f"Best route found: {best_path.genome}")
print(f"Cost of the best route: {tsp_cost(best_path.genome):.2f} km")

Generation 0: Cost of the best route: 1396.31 km
Generation 10000: Cost of the best route: 1345.54 km
Generation 20000: Cost of the best route: 1345.54 km
Generation 30000: Cost of the best route: 1345.54 km
Generation 40000: Cost of the best route: 1345.54 km
Generation 50000: Cost of the best route: 1345.54 km
Generation 60000: Cost of the best route: 1345.54 km
Generation 70000: Cost of the best route: 1345.54 km
Generation 80000: Cost of the best route: 1345.54 km
Generation 90000: Cost of the best route: 1345.54 km
Generation 100000: Cost of the best route: 1345.54 km
Generation 110000: Cost of the best route: 1345.54 km
Generation 120000: Cost of the best route: 1345.54 km
Generation 130000: Cost of the best route: 1345.54 km
Generation 140000: Cost of the best route: 1345.54 km


/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_58327/541863820.py:38: RuntimeWarning: overflow encountered in scalar divide
  if delta_fitness > 0 or random.random() < np.exp(delta_fitness / temperature):


Generation 150000: Cost of the best route: 1345.54 km
Generation 160000: Cost of the best route: 1345.54 km
Generation 170000: Cost of the best route: 1345.54 km
Generation 180000: Cost of the best route: 1345.54 km
Generation 190000: Cost of the best route: 1345.54 km
Best route found: [2, 0, 7, 1, 4, 3, 5, 6, 2]
Cost of the best route: 1345.54 km
